In [3]:
import numpy as np
import h5py
from pathlib import Path

In [4]:
import sys
sys.path.append("../")
from utils.config import labels

print(f"{labels=}")

labels=['A capella', 'Accelerating, revving, vroom', 'Accordion', 'Acoustic guitar', 'Afrobeat', 'Air brake', 'Air conditioning', 'Air horn, truck horn', 'Aircraft', 'Aircraft engine', 'Alarm', 'Alarm clock', 'Ambient music', 'Ambulance (siren)', 'Angry music', 'Animal', 'Applause', 'Arrow', 'Artillery fire', 'Babbling', 'Baby cry, infant cry', 'Baby laughter', 'Background music', 'Bagpipes', 'Bang', 'Banjo', 'Bark', 'Basketball bounce', 'Bass drum', 'Bass guitar', 'Bathtub (filling or washing)', 'Battle cry', 'Beatboxing', 'Bee, wasp, etc.', 'Beep, bleep', 'Bell', 'Bellow', 'Belly laugh', 'Bicycle', 'Bicycle bell', 'Bird', 'Bird flight, flapping wings', 'Bird vocalization, bird call, bird song', 'Biting', 'Bleat', 'Blender', 'Bluegrass', 'Blues', 'Boat, Water vehicle', 'Boiling', 'Boing', 'Boom', 'Bouncing', 'Bow-wow', 'Bowed string instrument', 'Brass instrument', 'Breaking', 'Breathing', 'Burping, eructation', 'Burst, pop', 'Bus', 'Busy signal', 'Buzz', 'Buzzer', 'Cacophony', 'Camer

# Counting the number of HDF5 Files in a directory

In [11]:
hdf5s_path : Path = Path("../hdf5s/pack_1.6m_hdf5s/train")


split_stats : dict = {}

for file in hdf5s_path.glob("*.h5"):
    print(f"{file=}")
    f = h5py.File(file)
    print(f['audio_name'])
    print()
    load_arr : np.array = np.array(f['valid'])
    print(f"{load_arr.any()}")
    print(f"{load_arr.shape=}")
    print("Number of valid files is ", np.sum(load_arr.astype(int)))
    print("Number of total entries is ", f"{load_arr.shape[0]}")
    # print(f"{}")
    split_stats[file] = {
        "num_audios" : np.sum(load_arr),
        "total_rows" : load_arr.shape[0]
    }
    break

total_num_audios : int = 0
total_rows : int = 0

for key in split_stats:
    total_num_audios += split_stats[key]['num_audios']
    total_rows += split_stats[key]['total_rows']

print(f"{total_num_audios=}")
print(f"{total_rows=}")




file=PosixPath('../hdf5s/pack_1.6m_hdf5s/train/audioset_0.h5')
<HDF5 dataset "audio_name": shape (41574,), type "|S20">

True
load_arr.shape=(41574,)
Number of valid files is  23197
Number of total entries is  41574
total_num_audios=23197
total_rows=41574


## Writing Attributes after the packing

For the HDF5 dataloader to work, I need to be able to set the attributes `target_names` under the `target` dataset, and I need to create an `hdf5_path` dataset linking to the *relative* path of the HDF5 file.

First, I need to test that this doesn't break anything by using a dummy HDF5.

In [42]:
import h5py

audios_num : int = 10
clip_samples : int = 100
classes_num : int = 10
sample_rate = 100

with h5py.File('../hdf5s/test.h5', 'w') as hf:
    hf.create_dataset('audio_name', shape=((audios_num,)), dtype='S20')
    wav_df = hf.create_dataset('waveform', shape=((audios_num, clip_samples)), dtype=np.int16)
    wav_df[:] = np.random.randn(audios_num, clip_samples)
    target_df = hf.create_dataset('target', shape=((audios_num, classes_num)), dtype=np.bool)
    hf.create_dataset('meta_csv_idx', shape=((audios_num,)), dtype=np.int32)
    hf.create_dataset('loaded_successfully', shape=((audios_num,)), dtype=np.bool)
    hf.attrs.create('sample_rate', data=sample_rate, dtype=np.int32)
    print(f"{hf.keys()=}")
    print(f"{hf['/audio_name']=}")
    print(f"{hf['/waveform']=}")
    print(f"{hf['/waveform'][0]=}")
    print(f"{hf['/target']=}")
    print(f"{hf['/target'][0]=}")
    print(f"{hf['/meta_csv_idx']=}")
    print(f"{hf['/loaded_successfully']=}")

hf.keys()=<KeysViewHDF5 ['audio_name', 'loaded_successfully', 'meta_csv_idx', 'target', 'waveform']>
hf['/audio_name']=<HDF5 dataset "audio_name": shape (10,), type "|S20">
hf['/waveform']=<HDF5 dataset "waveform": shape (10, 100), type "<i2">
hf['/waveform'][0]=array([ 0,  0,  0, -2,  0,  0,  0,  0,  0,  1, -1,  1,  0, -1,  0,  0,  2,
        0, -1, -1,  0,  1, -1,  0,  0, -1,  0,  1,  1,  0,  0,  0,  0,  0,
        2,  1,  0, -1, -2,  0,  0,  0,  0, -1,  0,  0, -1,  1,  1,  0,  0,
        1,  0,  1, -1,  0, -1, -1,  0,  1,  0, -1, -1,  0,  0,  0,  0,  0,
        1,  0, -1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,
        0,  2, -1,  1,  1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0],
      dtype=int16)
hf['/target']=<HDF5 dataset "target": shape (10, 10), type "|b1">
hf['/target'][0]=array([False, False, False, False, False, False, False, False, False,
       False])
hf['/meta_csv_idx']=<HDF5 dataset "meta_csv_idx": shape (10,), type "<i4">
hf['/loaded_successfully']=<HDF5

/tmp/ipykernel_2344001/2771089257.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  target_df = hf.create_dataset('target', shape=((audios_num, classes_num)), dtype=np.bool)
/tmp/ipykernel_2344001/2771089257.py:14: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  hf.create_dataset('loaded_successfully', shape=((audios_num,)), dtype=np.bool)


In [43]:
with h5py.File('../hdf5s/test.h5', 'a') as hf:
    print(f"{hf.keys()}")
    hf['/waveform'].attrs["target_names"] = ["something" , "other"]
    print(f"{hf['/waveform'][0]=}")
    print(f"{hf['/target'].attrs.keys()=}")

<KeysViewHDF5 ['audio_name', 'loaded_successfully', 'meta_csv_idx', 'target', 'waveform']>
hf['/waveform'][0]=array([ 0,  0,  0, -2,  0,  0,  0,  0,  0,  1, -1,  1,  0, -1,  0,  0,  2,
        0, -1, -1,  0,  1, -1,  0,  0, -1,  0,  1,  1,  0,  0,  0,  0,  0,
        2,  1,  0, -1, -2,  0,  0,  0,  0, -1,  0,  0, -1,  1,  1,  0,  0,
        1,  0,  1, -1,  0, -1, -1,  0,  1,  0, -1, -1,  0,  0,  0,  0,  0,
        1,  0, -1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,
        0,  2, -1,  1,  1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0],
      dtype=int16)
hf['/target'].attrs.keys()=<KeysViewHDF5 []>


In [44]:
with h5py.File('../hdf5s/test.h5', 'r') as hf:
    print(f"{hf['/audio_name']=}")
    print(f"{hf['/waveform']=}")
    print(f"{hf['/waveform'][0]=}")
    print(f"{hf['/target']=}")
    print(f"{hf['/target'][0]=}")
    print(f"{hf['/meta_csv_idx']=}")
    print(f"{hf['/loaded_successfully']=}")

hf['/audio_name']=<HDF5 dataset "audio_name": shape (10,), type "|S20">
hf['/waveform']=<HDF5 dataset "waveform": shape (10, 100), type "<i2">
hf['/waveform'][0]=array([ 0,  0,  0, -2,  0,  0,  0,  0,  0,  1, -1,  1,  0, -1,  0,  0,  2,
        0, -1, -1,  0,  1, -1,  0,  0, -1,  0,  1,  1,  0,  0,  0,  0,  0,
        2,  1,  0, -1, -2,  0,  0,  0,  0, -1,  0,  0, -1,  1,  1,  0,  0,
        1,  0,  1, -1,  0, -1, -1,  0,  1,  0, -1, -1,  0,  0,  0,  0,  0,
        1,  0, -1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1,  0,  0,
        0,  2, -1,  1,  1,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0],
      dtype=int16)
hf['/target']=<HDF5 dataset "target": shape (10, 10), type "|b1">
hf['/target'][0]=array([False, False, False, False, False, False, False, False, False,
       False])
hf['/meta_csv_idx']=<HDF5 dataset "meta_csv_idx": shape (10,), type "<i4">
hf['/loaded_successfully']=<HDF5 dataset "loaded_successfully": shape (10,), type "|b1">


### Indices for TRAINING Set

In [56]:
train_indices_path : str = "./hdf5s/train_indices.h5"

# with h5py.File(train_indices_path, 'w') as hf:
#     hf

hdf5s_path : Path = Path("../hdf5s")

split_stats : dict = {}

hdf5_paths : list[str] = []
index_in_hdf5_path : list[str] = []

intra_hdf5_idxs : list[np.array] = []
one_hot_targets : np.array = []
# really just going to give it the filename and let the dataloader handle the directory
hdf5_filenames : str = []

for file in hdf5s_path.glob("*.h5"):
    print(f"{file=}")
    print("==============================")
    # WARNING: ONLY SET TO 'r'
    # IF YOU SET IT TO W, it WILL MOST LIKELY OVERRIDE NO MATTER WHAT YOU DO
    with h5py.File(file, "r") as hf:
        loaded_successfully : np.array = np.array(hf['loaded_successfully'])
        only_successful : np.array = np.where(loaded_successfully)[0]
        all_targets : np.array = np.array(hf['target'])

        loaded_targets : np.array = all_targets[only_successful]
        intra_idxs : np.array = only_successful

        B : int = loaded_targets.shape[0]
        num_classes : int = len(labels) 
        print(f"{loaded_targets.shape=}")
        print(f"{only_successful.shape=}")
        assert loaded_targets.shape[0] == only_successful.shape[0]

        one_hot : np.array = np.zeros((B, num_classes))
        # print(f"{all_targets[0]}")
        # one_hot[:, all_targets] = 1
        one_hot_targets.append(one_hot)
        intra_hdf5_idxs.append(intra_idxs)
        hdf5_filenames.extend([file.name for x in range(B)])
    # break

print(f"------------------------------")
total_onehot : np.array = np.concatenate(one_hot_targets, axis=0)
total_idxs : np.array = np.concatenate(intra_hdf5_idxs, axis=0)

# yes, I know this should follow from the loop assertions
assert total_onehot.shape[0] == total_idxs.shape[0]
total_onehot = total_onehot.astype(bool)
print(f"{total_onehot.shape=}")
print(f"{total_idxs.shape=}")
assert total_onehot.dtype == np.bool

N : int = total_onehot.shape[0]
C : int = total_onehot.shape[1]
with h5py.File("./train_hdf5_idxs.h5", "w") as hf:
    hf.create_dataset('hdf5_filenames', shape=((N,)), dtype='S20')
    hf.create_dataset('index_in_hdf5', shape=((N,)), dtype=np.int32)
    hf.create_dataset('target', shape=((N, C)), dtype=np.bool)
    hf["/hdf5_filenames"][:] = hdf5_filenames
    hf["/index_in_hdf5"][:] = total_idxs
    hf['/target'][:] = total_onehot
    hf['target'].attrs['target_names'] = labels

file=PosixPath('../hdf5s/audioset_0.h5')
loaded_targets.shape=(41715, 527)
only_successful.shape=(41715,)
file=PosixPath('../hdf5s/audioset_1.h5')
loaded_targets.shape=(37934, 527)
only_successful.shape=(37934,)
file=PosixPath('../hdf5s/audioset_2.h5')
loaded_targets.shape=(30250, 527)
only_successful.shape=(30250,)
file=PosixPath('../hdf5s/audioset_3.h5')
loaded_targets.shape=(30468, 527)
only_successful.shape=(30468,)
file=PosixPath('../hdf5s/audioset_4.h5')
loaded_targets.shape=(30430, 527)
only_successful.shape=(30430,)
file=PosixPath('../hdf5s/audioset_5.h5')
loaded_targets.shape=(30390, 527)
only_successful.shape=(30390,)
file=PosixPath('../hdf5s/audioset_6.h5')
loaded_targets.shape=(31364, 527)
only_successful.shape=(31364,)
file=PosixPath('../hdf5s/audioset_7.h5')
loaded_targets.shape=(31723, 527)
only_successful.shape=(31723,)
file=PosixPath('../hdf5s/audioset_8.h5')
loaded_targets.shape=(31693, 527)
only_successful.shape=(31693,)
file=PosixPath('../hdf5s/audioset_9.h5')
loade

/tmp/ipykernel_2344001/1823834769.py:54: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert total_onehot.dtype == np.bool
/tmp/ipykernel_2344001/1823834769.py:61: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  hf.create_dataset('target', shape=((N, C)), dtype=np.bool)


In [58]:
# WARNING: ONLY USE 'r' HERE
with h5py.File("./train_hdf5_idxs.h5", "r") as hf:
    print(f"{hf['/hdf5_filenames'][:10]=}")
    print(f"{hf['/index_in_hdf5'][:10]=}")
    print(f"{hf['/target'][:10]=}")
    print(f"{hf['/target'].attrs['target_names']=}")

hf['/hdf5_filenames'][:10]=array([b'audioset_0.h5', b'audioset_0.h5', b'audioset_0.h5',
       b'audioset_0.h5', b'audioset_0.h5', b'audioset_0.h5',
       b'audioset_0.h5', b'audioset_0.h5', b'audioset_0.h5',
       b'audioset_0.h5'], dtype='|S20')
hf['/index_in_hdf5'][:10]=array([ 0,  3,  4,  5,  7,  8,  9, 10, 11, 12], dtype=int32)
hf['/target'][:10]=array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
hf['/target'].attrs['target_names']=array(['A capella', 'Accelerating, revving, vroom', 'Accordion',
       'Acoustic guitar', 'Afrobeat', 'Air brake', 'Air conditioning',
       'Air horn, truck horn', 'Aircraft', 'Aircraft engine', 'Alarm',
       'Alarm clock', 'Ambient music', 'Ambulance (s

### Indices for TRAINING Set

In [6]:
hdf5s_path : Path = Path("../eval_set.h5")

split_stats : dict = {}

hdf5_paths : list[str] = []
index_in_hdf5_path : list[str] = []

intra_hdf5_idxs : list[np.array] = []
one_hot_targets : np.array = []
# really just going to give it the filename and let the dataloader handle the directory
hdf5_filenames : str = []

print("==============================")
# WARNING: ONLY SET TO 'r'
# IF YOU SET IT TO W, it WILL MOST LIKELY OVERRIDE NO MATTER WHAT YOU DO
with h5py.File(hdf5s_path, "r") as hf:
    loaded_successfully : np.array = np.array(hf['loaded_successfully'])
    only_successful : np.array = np.where(loaded_successfully)[0]
    all_targets : np.array = np.array(hf['target'])

    loaded_targets : np.array = all_targets[only_successful]
    intra_idxs : np.array = only_successful

    B : int = loaded_targets.shape[0]
    num_classes : int = len(labels) 
    print(f"{loaded_targets.shape=}")
    print(f"{only_successful.shape=}")
    assert loaded_targets.shape[0] == only_successful.shape[0]

    one_hot : np.array = np.zeros((B, num_classes))
    # print(f"{all_targets[0]}")
    # one_hot[:, all_targets] = 1
    one_hot_targets.append(one_hot)
    intra_hdf5_idxs.append(intra_idxs)
    hdf5_filenames.extend([hdf5s_path.name for x in range(B)])
    # break

print(f"------------------------------")
total_onehot : np.array = np.concatenate(one_hot_targets, axis=0)
total_idxs : np.array = np.concatenate(intra_hdf5_idxs, axis=0)

# yes, I know this should follow from the loop assertions
assert total_onehot.shape[0] == total_idxs.shape[0]
total_onehot = total_onehot.astype(bool)
print(f"{total_onehot.shape=}")
print(f"{total_idxs.shape=}")
assert total_onehot.dtype == np.bool

N : int = total_onehot.shape[0]
C : int = total_onehot.shape[1]
with h5py.File("./eval_hdf5_idxs.h5", "w") as hf:
    hf.create_dataset('hdf5_filenames', shape=((N,)), dtype='S20')
    hf.create_dataset('index_in_hdf5', shape=((N,)), dtype=np.int32)
    hf.create_dataset('target', shape=((N, C)), dtype=np.bool)
    hf["/hdf5_filenames"][:] = hdf5_filenames
    hf["/index_in_hdf5"][:] = total_idxs
    hf['/target'][:] = total_onehot
    hf['target'].attrs['target_names'] = labels

loaded_targets.shape=(15955, 527)
only_successful.shape=(15955,)
------------------------------
total_onehot.shape=(15955, 527)
total_idxs.shape=(15955,)


/tmp/ipykernel_1827981/3878292931.py:47: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  assert total_onehot.dtype == np.bool
/tmp/ipykernel_1827981/3878292931.py:54: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  hf.create_dataset('target', shape=((N, C)), dtype=np.bool)


In [7]:
# WARNING: ONLY USE 'r' HERE
with h5py.File("./eval_hdf5_idxs.h5", "r") as hf:
    print(f"{hf['/hdf5_filenames'][:10]=}")
    print(f"{hf['/index_in_hdf5'][:10]=}")
    print(f"{hf['/target'][:10]=}")
    print(f"{hf['/target'].attrs['target_names']=}")

hf['/hdf5_filenames'][:10]=array([b'eval_set.h5', b'eval_set.h5', b'eval_set.h5', b'eval_set.h5',
       b'eval_set.h5', b'eval_set.h5', b'eval_set.h5', b'eval_set.h5',
       b'eval_set.h5', b'eval_set.h5'], dtype='|S20')
hf['/index_in_hdf5'][:10]=array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10], dtype=int32)
hf['/target'][:10]=array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])
hf['/target'].attrs['target_names']=array(['A capella', 'Accelerating, revving, vroom', 'Accordion',
       'Acoustic guitar', 'Afrobeat', 'Air brake', 'Air conditioning',
       'Air horn, truck horn', 'Aircraft', 'Aircraft engine', 'Alarm',
       'Alarm clock', 'Ambient music', 'Ambulance (siren)', 'Angry music',
    